In [1]:
# Inspired by https://github.com/srush/Tensor-Puzzles
from tinygrad import Tensor
import numpy as np

In [ ]:
# These puzzles are about broadcasting. Know this rule.
# Each puzzle needs to be solved in 1 line (<80 columns) of code.
# You are allowed @, arithmetic, comparison, shape, any indexing (e.g. a[:j], a[:, None], a[arange(10)]), and previous puzzle functions.
# You are not allowed anything else. No view, sum, take, squeeze, tensor.

In [4]:
# arange
print(Tensor.arange(3))

<Tensor <LB METAL (3,) int (<BinaryOps.ADD: 1>, None)> on METAL with grad None>


In [3]:
# Example of broadcasting.
examples = [(Tensor.arange(4), Tensor.arange(5)[:, None]) ,
            (Tensor.arange(3)[:, None], Tensor.arange(2))]

print([a + b for a,b in examples])

[<Tensor <LB METAL (5, 4) int (<BinaryOps.ADD: 1>, None)> on METAL with grad None>, <Tensor <LB METAL (3, 2) int (<BinaryOps.ADD: 1>, None)> on METAL with grad None>]


In [5]:
# where
examples = [(Tensor([False]), Tensor([10]), Tensor([0])),
            (Tensor([False, True]), Tensor([1, 1]), Tensor([-10, 0])),
            (Tensor([False, True]), Tensor([1]), Tensor([-10, 0])),
            (Tensor([[False, True], [True, False]]), Tensor([1]), Tensor([-10, 0])),
            (Tensor([[False, True], [True, False]]), Tensor([[0], [10]]), Tensor([-10, 0])),
           ]

[q.where(a, b).numpy() for q, a, b in examples]

[array([0], dtype=int32),
 array([-10,   1], dtype=int32),
 array([-10,   1], dtype=int32),
 array([[-10,   1],
        [  1,   0]], dtype=int32),
 array([[-10,   0],
        [ 10,   0]], dtype=int32)]

In [13]:
# Puzzle 1 - ones
def ones(i) -> Tensor:
  # return Tensor.arange(i) - Tensor.arange(i) + 1
  return Tensor.where(Tensor.arange(i) > -1, 1, 0)

assert np.array_equal(ones(5).numpy(), np.ones(5))

In [7]:
# Puzzle 2 - sum
def sum(a: Tensor) -> Tensor:
  return a @ ones(a.shape[0])

a = Tensor([1, 2, 4, 5])
assert sum(a).numpy() == a.sum().numpy()

In [8]:
# Puzzle 3 - outer
def outer(a: Tensor, b: Tensor) -> Tensor:
  return a[:, None] * b[None, :]

a = Tensor.randint(6)
b = Tensor.randint(3)
assert np.array_equal(outer(a, b).numpy(), np.outer(a.numpy(), b.numpy()))

In [56]:
# Puzzle 4 - diag
def diag(a: Tensor) -> Tensor:
  return a[Tensor.arange(a.shape[0]), Tensor.arange(a.shape[0])]

a = Tensor.randint((5, 5))
assert np.array_equal(diag(a).numpy(), np.diag(a.numpy()))

In [57]:
# Puzzle 5 - eye
def eye(i) -> Tensor:
  # return Tensor.where(Tensor.arange(i)[:, None] == Tensor.arange(i)[None, :], 1, 0)
  return Tensor.arange(i)[:, None] == Tensor.arange(i)

assert np.array_equal(eye(5).numpy(), np.eye(5))

In [61]:
# Puzzle 6 - triu
def triu(j: Tensor) -> Tensor:
  return Tensor.where(Tensor.arange(j)[:, None] <= Tensor.arange(j)[None, :], 1, 0)

def triu(a: Tensor, j: int) -> Tensor:
  # return Tensor.where(Tensor.arange(a.shape[0])[:, None] <= (Tensor.arange(a.shape[0])[None, :] - j), a, 0)
  return Tensor.where(Tensor.arange(a.shape[0])[:, None] <= (Tensor.arange(a.shape[0]) - j), a, 0) # cleaner

a = Tensor.randint((5, 5))
assert np.array_equal(triu(a, 3).numpy(), np.triu(a.numpy(), 3))
assert np.array_equal(triu(a, -1).numpy(), np.triu(a.numpy(), -1))

In [65]:
# Puzzle 7 - cumsum
def cumsum(a: Tensor) -> Tensor:
  return a @ Tensor.where(Tensor.arange(a.shape[0])[:, None] <= Tensor.arange(a.shape[0]), 1, 0)

a = Tensor.randint(5)
assert np.array_equal(cumsum(a).numpy(), np.cumsum(a.numpy()))

In [110]:
# Puzzle 8 - diff
def diff(a: Tensor, i:int) -> Tensor:
  return a[:i] - Tensor.where(Tensor.arange(i) != 0, a[Tensor.arange(i) - 1], 0)

a = Tensor.randint(5)
# assert np.array_equal(diff(a, 2).numpy(), np.diff(a.numpy(), 2))

In [117]:
# Puzzle 9 - vstack
def vstack(a: Tensor, b: Tensor):
  return Tensor.where(Tensor.arange(2)[:, None] == ones(a.shape[0]), b, a)

a = Tensor.randint(5)
b = Tensor.randint(5)
assert np.array_equal(vstack(a, b).numpy(), np.vstack((a.numpy(), b.numpy())))

In [123]:
# Puzzle 10 - roll
def roll(a: Tensor, i: int) -> Tensor:
  return a[(Tensor.arange(i) + 1) % i] # tensor mod not supported in tinygrad yet

a = Tensor.randint(5)
assert np.array_equal(roll(a, 2).numpy(), np.roll((a.numpy(), 2)))

TypeError: unsupported operand type(s) for %: 'Tensor' and 'int'

In [141]:
# Puzzle 11 - flip
def flip(a: Tensor, i: int) -> Tensor:
  return Tensor.where(Tensor.arange(a.shape[0]) < i, a[i - Tensor.arange(a.shape[0]) - 1], a[Tensor.arange(a.shape[0])])

a = Tensor.randint(5)
print(a.numpy())
print(flip(a, 3).numpy())

[7 5 2 3 1]
[2 5 7 3 1]


In [152]:
# Puzzle 12 - compress
def compress(g: Tensor, v: Tensor, i: int) -> Tensor:
  return v @ Tensor.where(g[:, None], Tensor.arange(i) == (cumsum(1 * g) - 1)[:, None], 0) # what?

v = Tensor([[1, 2], [3, 4], [5, 6]])
g = Tensor([0, 1])
print(compress(g, v, 1).numpy())

[[2]
 [4]
 [6]]


In [187]:
# Puzzle 13 - pad_to
def pad_to(a: Tensor, i: int) -> Tensor:
  return Tensor.where(Tensor.arange(i) <= i, a[Tensor.arange(i)], 0)


a = Tensor.randint(5)
print(a.numpy())
print(pad_to(a, 3).numpy())
print(pad_to(a, 7).numpy())

[2 5 3 2 7]
[2 5 3]
[2 5 3 2 7 0 0]


In [209]:
def sequence_mask(a: Tensor, i: int) -> Tensor:
  return Tensor.where((a[:, None] * ones(i)[None, :]) > Tensor.arange(i), True, False)

i = 7
a = Tensor.randint(10, high=i+1)
print(a.numpy())
print(sequence_mask(a, i).numpy())

[5 3 0 7 0 4 2 3 3 5]
[[ True  True  True  True  True False False]
 [ True  True  True False False False False]
 [False False False False False False False]
 [ True  True  True  True  True  True  True]
 [False False False False False False False]
 [ True  True  True  True False False False]
 [ True  True False False False False False]
 [ True  True  True False False False False]
 [ True  True  True False False False False]
 [ True  True  True  True  True False False]]


In [345]:
# Puzzle 15 - bincount
def bincount(a: Tensor) -> Tensor:
  return ones(a.shape[0]) @ eye(a.shape[0])[a]

n = 10
a = Tensor.randint(n, high=n)
assert np.array_equal(bincount(a).numpy(), np.bincount(a.numpy()))

In [361]:
# Puzzle 16 - scatter_add
def scatter_add(v: Tensor, link: Tensor, j: int) -> Tensor:
  return v @ eye(j)[link]


v = Tensor([5, 1, 7, 2, 3, 2, 1, 3])
link = Tensor([0, 0, 1, 0, 2, 2, 3, 3])
j = 4
print(scatter_add(v, link, j).numpy())

[8 7 5 4]


In [376]:
# Puzzle 17 - flatten
def flatten(a: Tensor, i: int, j: int) -> Tensor:
  return a[Tensor.arange(i*j) / j, Tensor.arange(i*j) % j] # tensor mod not supported in tinygrad yet

a = Tensor.randint((5, 5))
print(flatten(a, 5, 5).numpy())
assert np.array_equal(flatten(a, 5, 5).numpy(), a.numpy().flatten())

TypeError: unsupported operand type(s) for %: 'Tensor' and 'int'

In [382]:
# Puzzle 18 - linspace
def linspace(i: int, j: int, n: int) -> Tensor:
  return i + (j - i) * Tensor.arange(n) / max(1, n -1)

print(linspace(2.0, 3.0, 5).numpy())

[2.   2.25 2.5  2.75 3.  ]


In [406]:
# Puzzle 19 - heaviside
def heaviside(a: Tensor, b: Tensor) -> Tensor:
  return Tensor.where(a == 0, b, (a > 0) * 1)

a = Tensor.randint(10)
b = Tensor.randint(10)
assert np.array_equal(heaviside(a, b).numpy(), np.heaviside(a.numpy(), b.numpy()))

In [474]:
# Puzzle 20 - repeat (1d)
def repeat(a: Tensor, n: int) -> Tensor:
  return Tensor.where(ones(n)[:, None], a, 0)

a = Tensor.randint(5)
print(a.numpy())
print(repeat(a, 3).numpy())

[5 3 6 0 6]
[[5 3 6 0 6]
 [5 3 6 0 6]
 [5 3 6 0 6]]


In [482]:
# Puzzle 21 - bucketize
def bucketize(v: Tensor, boundaries: Tensor) -> Tensor:
  return (1 * (v[:, None] >= boundaries)) @ ones(boundaries.shape[0])

v = Tensor([3, 6, 9])
boundaries = Tensor([1, 3, 5, 7, 9])
print(bucketize(v, boundaries).numpy())

[1 3 4]
